<a href="https://colab.research.google.com/github/jeraldflowers/Models-HuggingFace/blob/main/Template_Gradio_Demos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to Gradio

Gradio offers two different APIs depending on the level of detail you are looking for:

- `gradio.Interface`: A high-level API that allows you to create ML demos simply by providing a list of inputs and outputs.

- `gradio.Blocks`: Low-level API that allows full control over data flows and application layout. Very complex multi-step applications can be created using Blocks (as if they were building blocks).

We will start by using `Interface` and at the end we will show an example of `Blocks`.

## Install Gradio

In [ ]:
!pip install gradio

Example using a greet function that has `text` as input and `text` as output.

In [ ]:
import gradio as gr

def greets(name):
  return "Hello " + name

demo = gr.Interface(fn=greets, inputs="text", outputs="text")

demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

The `gr.Interface` class is an easy way to create demos that can be anything from a calculator to a speech recognition application. 

It is initialized with three required parameters:


*   `fn`: function.

*   `inputs`: which component(s) to use for function inputs, eg "text", "image" or "audio"
*   `outputs`: which component(s) to use for the function's outputs, e.g. "text", "image" or "label"


Gradio includes more than 20 different components, most of which can be used as inputs and outputs. In the documentation is the [complet list](https://gradio.app/docs/#components).

Ejemplo 2:

In [ ]:
import gradio as gr

def greets(name):
  return "Hello " + name

demo = gr.Interface(fn=greets, inputs=gr.Textbox(linear=10, placeholder="Tell me your name"), outputs="text")

demo.launch()

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:44: UserWarning: You have unused kwarg parameters in Textbox, please remove them: {'linear': 10}
  f"You have unused kwarg parameters in {cls}, please remove them: {kwargs}"


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Demos for image classification

**Example 1:** demo with loaded model of TensorFlow applications.

In [ ]:
import tensorflow as tf

inception_net = tf.keras.applications.MobileNetV2()

14536120/14536120 [==============================] - 0s 0us/step


In [ ]:
import requests

# Obteniendo las labels de "https://git.io/JJkYN"
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

def image_classifier(inp):
  inp = inp.reshape((-1,224,224,3))
  inp = tf.keras.applictaion.mobilenet_v2.preprocess_input(inp)
  prediction = inception_net.predict(inp).flatten()
  confidences = {labels[i]: float(prediction[i]) for i in range(1000)}
  return confidences

In [ ]:
import gradio as gr

demo = gr.Interface(fn=image_classifier,
                    inputs=gr.Image(shape=(224,224)),
                    outputs=gr.Label(num_top_classes=3))

demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

**Example 2**: with loaded model Hugging Face Hub. 

In [ ]:
import gradio as gr

title = "My first Demo with Hugging Face"
description = "This is a demo as a example"

gr.Interface.load(
    "huggingface/microsoft/swin-tiny-patch4-window7-224",
    inputs=gr.Image(label="Load an image here"),
    title=title,
    description=description
).launch()

Fetching model from: https://huggingface.co/microsoft/swin-tiny-patch4-window7-224
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Transcription of audio to text demo

We used `pipeline` to load a model for automatic speech recognition in Spanish from the Hugging Face Hub.

The demo automatically transcribes what we recorded.

In [ ]:
%%capture
!pip install transformers
!pip install torch

In [ ]:
from transformers import pipeline
import gradio as gr

model = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-large-xlsr-53-spanish")

def transcribe(audio):
  text = model(audio)["text"]
  return text

In [ ]:
gr.Interface(
    fn=transcribe,
    inputs=[gr.Audio(source="microphone", type="filepath")],
    outputs=["textbox"],
).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Blocks

We created a demo that receives two models. You can transcribe a voice and you can also classify the sentiment of a text. 

In [ ]:
%%capture
!pip install gradio
!pip install transformers
!pip install torch 

In [ ]:
import gradio as gr
from transformers import pipeline

trans = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-large-xlsr-53-spanish")
classifier = pipeline("text-classification", model="pysentimiento/robertuito-sentiment-analysis")

Downloading:   0%|          | 0.00/925 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/435M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/334 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/858k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
def audio_to_text(audio):
  text = trans(audio)["text"]
  return text

def text_to_sentiment(text):
  return classifier(text)[0]["label"]

In [ ]:
demo = gr.Blocks()

with demo:
  gr.Markdown("Demo for the practice")
  audio = gr.Audio(source="microphone", type="filepath")
  texto = gr.Textbox()
  b1 = gr.Button("Transcribe")
  b1.click(audio_to_text, inputs=audio, outputs=texto)

  label = gr.Label()
  b2 = gr.Button("classify the feeling")
  b2.click(text_to_sentiment, inputs=texto, outputs=label)

demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Let's make a blocks app a bit more interesting.

In [ ]:
demo = gr.Blocks()

with demo:
  gr.Markdown("This is the second demo with Blocks")
  with gr.Tabs():
    with gr.TabItem("Transcribe audio in Spanish"):
      with gr.Row():
        audio = gr.Audio(source="microphone", type="filepath")
        transcription = gr.Textbox()
      b1 = gr.Button("Transcribe")
    
    with gr.TabItem("Sentiment Analysis in Spanish"):
      with gr.Row():
        texto = gr.Textbox()
        label = gr.Label()
      b2 = gr.Button("Sentiment")

    b1.click(audio_to_text, inputs=audio, outputs=transcription)
    b2.click(text_to_sentiment, inputs=texto, outputs=label)

demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>